In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
from zipfile import ZipFile
zf = ZipFile(r'./dados/archive.zip')
dados = pd.read_excel(zf.open('Kaggle_Sirio_Libanes_ICU_Prediction.xlsx'))

In [5]:
dados.describe()

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,...,RESPIRATORY_RATE_DIFF,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL,ICU
count,1925.000000,1925.000000,1925.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,1920.000000,...,1177.000000,1231.000000,1239.000000,1240.000000,1240.000000,1240.000000,1177.000000,1231.000000,1239.000000,1925.000000
mean,192.000000,0.467532,0.368831,0.108333,0.028125,0.097917,0.019792,0.128125,0.046875,0.213021,...,-0.703683,-0.770338,-0.887196,-0.786997,-0.715950,-0.817800,-0.719147,-0.771327,-0.886982,0.267532
std,111.168431,0.499074,0.482613,0.310882,0.165373,0.297279,0.139320,0.334316,0.211426,0.409549,...,0.482097,0.319001,0.296147,0.324754,0.419103,0.270217,0.446600,0.317694,0.296772,0.442787
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000
25%,96.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000
50%,192.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-1.000000,-0.976190,-0.979798,-1.000000,-0.984944,-0.989822,-1.000000,-0.975924,-0.980333,0.000000
75%,288.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.647059,-0.595238,-0.878788,-0.645482,-0.522176,-0.662529,-0.634409,-0.594677,-0.880155,1.000000
max,384.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
dados['ICU'].value_counts()

0    1410
1     515
Name: ICU, dtype: int64

In [7]:
# Removendo os valores nulos
dados_limpos = dados.dropna()

# Para pegar somente as tabelas que possuem valores numéricos iremos utilizar dados.describe().columns
x = dados_limpos[dados_limpos.describe().columns].drop(['ICU'], axis=1)
y = dados_limpos['ICU']

In [8]:
x.head()

,PATIENT_VISIT_IDENTIFIER,AGE_ABOVE65,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,...,HEART_RATE_DIFF,RESPIRATORY_RATE_DIFF,TEMPERATURE_DIFF,OXYGEN_SATURATION_DIFF,BLOODPRESSURE_DIASTOLIC_DIFF_REL,BLOODPRESSURE_SISTOLIC_DIFF_REL,HEART_RATE_DIFF_REL,RESPIRATORY_RATE_DIFF_REL,TEMPERATURE_DIFF_REL,OXYGEN_SATURATION_DIFF_REL
4,0,1,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.114504,0.176471,-0.238095,-0.818182,-0.389967,0.407558,-0.230462,0.096774,-0.242282,-0.814433
6,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
8,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-0.923664,-0.764706,-0.880952,-1.000000,-0.906832,-0.831132,-0.940967,-0.817204,-0.882574,-1.000000
9,1,1,1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.114504,0.764706,0.142857,-0.797980,0.315690,0.200359,-0.239515,0.645161,0.139709,-0.802317
14,2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.419847,-0.705882,-0.500000,-0.898990,-0.612422,-0.343258,-0.576744,-0.695341,-0.505464,-0.900129


In [9]:
from sklearn.linear_model import LogisticRegression

modelo = LogisticRegression()
modelo.fit(x,y)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [10]:
x.iloc[4]

PATIENT_VISIT_IDENTIFIER           2.000000
AGE_ABOVE65                        0.000000
GENDER                             0.000000
DISEASE GROUPING 1                 0.000000
DISEASE GROUPING 2                 0.000000
                                     ...   
BLOODPRESSURE_SISTOLIC_DIFF_REL   -0.343258
HEART_RATE_DIFF_REL               -0.576744
RESPIRATORY_RATE_DIFF_REL         -0.695341
TEMPERATURE_DIFF_REL              -0.505464
OXYGEN_SATURATION_DIFF_REL        -0.900129
Name: 14, Length: 228, dtype: float64

In [11]:
# Testando a precisão do modelo. Importante passar o x.iloc dentro do colchetes pois se não vai passar os valores das colunas somente, então o modelo não entenderá o que é para fazer
modelo.predict([x.iloc[4]])

array([1], dtype=int64)

In [12]:
y.iloc[4]

1

Podemos observar que o modelo teve sucesso em prever que de fato o paciente em questão foi para a UTI

In [13]:
#display(print('O modelo acertou em: %d dos casos' % (sum(modelo.predict(x[:20]) == y[:20]) )))
display(print('O modelo acertou em: {} dos casos'.format(sum(modelo.predict(x[:20]) == y[:20]) )))
modelo.predict(x[:20]) == y[:20]

O modelo acertou em: 20 dos casos


None

4     True
6     True
8     True
9     True
14    True
19    True
22    True
24    True
29    True
34    True
36    True
39    True
44    True
49    True
50    True
54    True
59    True
64    True
69    True
70    True
Name: ICU, dtype: bool

In [14]:
#display(print('O modelo acertou em: %d dos casos' % (sum(modelo.predict(x[:20]) == y[:20]) )))
print('O modelo acertou em: {} dos casos e errou em {}'.format(sum(modelo.predict(x) == y),sum(modelo.predict(x) != y) ))
print('A acuracia do modelo foi de {:.2%} '.format(sum(modelo.predict(x) == y)/len(y)))

O modelo acertou em: 497 dos casos e errou em 48
A acuracia do modelo foi de 91.19% 


Separando os dados em treino e teste com a ferramenta train_train_split

In [41]:
from sklearn.model_selection import train_test_split

# Stratificar é manter a proporção com base em alguma coluna, por exemplo, no nosso caso irá manter a proporção de 0 e 1 com base na coluna y
x_train, x_test, y_train, y_test = train_test_split(x,y, stratify = y, random_state = 73246)

In [42]:
import warnings
#ignorar os avisos do python
warnings.filterwarnings('ignore')

from sklearn.metrics import accuracy_score
# Definindo o modelo
modelo = LogisticRegression()

# Treinando o modelo
modelo.fit(x_train, y_train)

# Fazendo as previsões para os dados de teste
yhat = modelo.predict(x_test)

# Comparando o que foi previsto com os valores corretos
print('{:.2%}'.format(accuracy_score(y_test, yhat)))

83.21%


In [44]:
print(y_train.value_counts(normalize = True))
print()
print(y_test.value_counts(normalize = True))

0    0.514706
1    0.485294
Name: ICU, dtype: float64

0    0.518248
1    0.481752
Name: ICU, dtype: float64


# Desafio 5: Decida quanto você vai escolher para treino e quanto para teste? 

O parâmetro utilizado é o **test_size** e o mais comum é utilizar 2/3 para os dados de treino e apenas 1/3 para dados de teste.

In [45]:
from sklearn.model_selection import train_test_split

# Stratificar é manter a proporção com base em alguma coluna, por exemplo, no nosso caso irá manter a proporção de 0 e 1 com base na coluna y
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 1/3, stratify = y, random_state = 73246)

# Desafio 6: Ler a documentação da Regressão Logistica

[LINK PARA A DOCUMENTAÇÃO LOGISTIC REGRESSION](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)